In [90]:
import re
import pandas as pd
import numpy as np
import requests
from lxml import html, etree

## https://news.bitcoin.com/


In [ ]:
## http://docs.python-guide.org/en/latest/scenarios/scrape/
##

class news_bitcoin_parser():
    def __init__(self):
        self.link = 'https://news.bitcoin.com/'
        self.save_dir = "news_bitcoin/"
    
    def read_article_text(self,article_href):
        article_page = requests.get(article_href)
        article_html = html.fromstring(article_page.content)
        article_content = article_html.xpath('//div[@class="td-post-content"]')
        text = etree.tostring(article_content[0])
        text = text.decode("utf-8")  
        return text
    
    def clean_text (self,article_text):
        article_text = re.sub("<.*?>","",article_text)
        article_text = re.sub("&#[0-9]+;|\n"," ",article_text)
        article_text = re.sub(" {1,}"," ",article_text)

        return article_text

    def parse_page(self,page_num, verbose = True):
        page_link = self.link
        if page_num>1:
            page_link = self.link + "/page/" + str(page_num)
        if page_num<=0:
            return 0
        page = requests.get(page_link)
        page_html = html.fromstring(page.content)
        
        news = page_html.xpath('//div[@class="td-container td-pb-article-list"]')
        articles = news[0].xpath('.//div[@class="item-details"]')

        ## for debugging
        self.page = page
        self.page_html = page_html
        self.articles = articles 
        if verbose: print ('read page ' + str(page_num))
        
        for i_article in articles:
            article_name = i_article.xpath(".//a/@title")[0]
            article_time = i_article.xpath(".//time/@datetime")[0]
            article_date = article_time[0:10]
            article_href = i_article.xpath(".//a/@href")[0]
            article_text = self.read_article_text(article_href)
            
            ## cleaning article from special symbols
            article_text = self.clean_text(article_text)
                        
            ## saving article
            fn = re.sub("[^a-zA-Z0-9 ]","",article_name[0:20])
            file_name = self.save_dir + article_date + "_" + 'p_' + '{num:03d}'.format(num=page_num) + "_" + fn + ".txt"
            if verbose: print (file_name)
            f = open(file_name,'w')
            f.write(article_name + "\n")
            f.write(article_time+ "\n")
            f.write(article_text)
            f.close()
        print ("page " + str(page_num) + " parsed")

            
bc_parser = news_bitcoin_parser()

for i in range(1,619):
    bc_parser.parse_page(i,verbose=False)

In [280]:
 requests.get(bc_parser.link)

<Response [200]>

In [288]:
bc_parser.parse_page(3,verbose=True)

read page 3
news_bitcoin/2018-03-25_p_3_This Week in Bitcoin.txt
news_bitcoin/2018-03-25_p_3_Crypto Mania Peaked .txt
news_bitcoin/2018-03-25_p_3_Venezuelans to Buy H.txt
news_bitcoin/2018-03-25_p_3_Tennessee Passes Bil.txt
news_bitcoin/2018-03-25_p_3_Reddit Removes Bitco.txt
news_bitcoin/2018-03-25_p_3_Cryptocurrency Has A.txt
news_bitcoin/2018-03-25_p_3_Markets Update: Trad.txt
news_bitcoin/2018-03-25_p_3_Hydro-Quebec Turns D.txt
news_bitcoin/2018-03-25_p_3_Bitcoin Ransomware A.txt
page 3 parsed


In [297]:
etree.tostring(bc_parser.articles[6])

i_article = bc_parser.articles[6]
article_href = i_article.xpath(".//a/@href")[0]

aa = read_article_text(article_href)
aa = bc_parser.clean_text(aa)
aa
    

' Market Updates 4 days ago | By Jamie Redman - | 6111 Markets Update: Trader Sentiment Optimistic But Signals Still Unclear Cryptocurrency markets are bouncing around in value over the course of the weekend as the entire crypto-market capitalization currently rests at $329 billion USD. BTC/USD markets briefly jumped above the $9K region on Sunday, March 25 but dropped an hour later to a low of $8,398. Most of the digital currencies within the crypto-economy have followed suit with BTC as most coins have lost 3-5 percent in value during the late night (EDT) trading sessions. Also read: Slush Pool Mines the First ASIC Boost Block Sparking More Debate Crypto Markets Hold Steady As Traders Wait for the Next Big Move The past few days most cryptocurrencies have been somewhat stable as they have been following the same triangular sideways pattern as BTC/USD markets. At the time of publication BTC s price is hovering above the $8,540 range as global markets are swapping roughly $5.4Bn worth 

In [294]:
article_href

'https://news.bitcoin.com/markets-update-trader-sentiment-optimistic-but-signals-still-unclear/'

KeyError: '03d'